In [ ]:
include("src/botmarket.jl")

using Parameters
using Statistics
using Distributions
using Plots
using StatsPlots

# Standard library
using Random

In [ ]:
price_expr = :(rand(Truncated(Normal(last_price, last_price/100), 0.01, 400)))
order_expr = :(rand([SellLimitOrder, BuyLimitOrder]))
quantity_expr = :(max_quantity == 0 ? 0 : rand(1:Int(max_quantity)));

In [ ]:
n_iters = 300
starting_cash = 4000
n_stocks = 40
n_traders = 100

stock = DoubleAuctionMarket("NOK")
stock.last_price = 95
prices = Array{Float64,1}()
inv = [SingleAssetInvestor(
        "Player " * string(n), starting_cash, n_stocks
        , price_expr=price_expr, order_expr=order_expr, quantity_expr=quantity_expr
        ) for n in 1:n_traders]


for n in 1:n_iters
    println("Placing...")
    for inv in inv
        place!(inv, stock)
    end
    println("Clearing...")
    clear!(stock)
    push!(prices, stock.last_price)

    if n < n_iters
        cancel!(stock)
    end
end

## Update

In [ ]:
n_iters = 300
starting_cash = 6000
n_stocks = 40
n_traders = 100

stock = DoubleAuctionMarket("NOK")
stock.last_price = 95
prices = Array{Float64,1}()
inv = [SingleAssetInvestor(
        "Player " * string(n), starting_cash, n_stocks
        , price_expr=price_expr, order_expr=order_expr, quantity_expr=quantity_expr
        ) for n in 1:n_traders]


for n in 1:n_iters
    println("Placing...")
    for inv in shuffle(inv)
        print(stock.last_price)
        place!(inv, stock)
        clear!(stock)
        print("Price: $(stock.last_price)")
    end
    push!(prices, stock.last_price)

    if n < n_iters
        cancel!(stock)
    end
end

## Plotting

In [ ]:
line_prices = plot(prices, label="", title="Last prices", xlabel="Iterations")
bar_positions = groupedbar(hcat(map(x-> x.cash, inv), map(x-> x.position * stock.last_price, inv))
    , bar_position = :stack, bar_width=1, labels=["cash", "value of stock position"], color = [:green :blue],
    title="Positions (at the end of the simulation)", xlabel="trader id")

plot(line_prices, bar_positions, layout = grid(2,1,heights=[0.3,0.3]), size=[700,1000])

In [ ]:
sell_prices = unique(map(x -> x.price, stock.sell_limit_orders))
sell_quantities = [-sum(map(x-> x.quantity, filter(x -> x.price ==price, stock.sell_limit_orders))) for price in sell_prices]

buy_prices = unique(map(x -> x.price, stock.buy_limit_orders))
buy_quantities = [sum(map(x -> x.quantity, filter(x -> x.price ==price, stock.buy_limit_orders))) for price in buy_prices]

bar(
    sell_prices, 
    sell_quantities, 
    orientation=:horizontal, bar_width = 0.01, color=:red, label="Sell", linewidth=0, xlabel="Quantity (negative for sell)", ylabel="Price")

bar!(
    buy_prices, 
    buy_quantities, 
    orientation=:horizontal, bar_width = 0.01, color=:green, label="Buy", linewidth=0)
vline!([0], color=:black, label="")
title!("Order book")

In [ ]:
n_iters = 1
starting_cash = 4000
n_stocks = 40
n_traders = 100

stock = DoubleAuctionMarket("NOK")
stock.last_price = 90

prices = Array{Float64,1}()
inv = [Investor(
        "Player " * string(n), starting_cash, n_stocks
        , price_expr=price_expr, order_expr=order_expr, quantity_expr=quantity_expr
        ) for n in 1:n_traders]

for inv in inv
    place!(inv, stock)
end



scatter(
    map(x-> x.quantity, stock.buy_limit_orders),
    map(x-> x.price, stock.buy_limit_orders),
    label="Buy", color=:green
)
scatter!(
    map(x-> -x.quantity, stock.sell_limit_orders),
    map(x-> x.price, stock.sell_limit_orders),
    label="Sell", color=:red
)
vline!([0], color="black", label="")
ps, qs = clear!(stock)

hline!([minimum(ps)], color=:green, label="Min price")
hline!([stock.last_price], color=:blue, label="Mean price")
hline!([maximum(ps)], color=:red, label="Max price")